In [3]:
#This is the code for Table4.6 in Chapter4.3.2:The implementation and analysis of the strategy
#After modifying the dataset file path, it can be directly reproduce the results.

#!!! IMPORTANT REMINDER !!! 
#!!! IMPORTANT REMINDER !!! 
#!!! IMPORTANT REMINDER !!! 
# Before running this code, make sure to modify the input dataset path in the # === 1. Load and preprocess data ===



import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from hmmlearn.hmm import GaussianHMM

# === 1. Load and preprocess data ===

#!!! IMPORTANT REMINDER !!! 
#!!! IMPORTANT REMINDER !!! 
#!!! IMPORTANT REMINDER !!! 
# !!! Modify "C:/Users/ZhangYinhang/ES_F_data.csv"  to your save path in the below !!!
df = pd.read_csv("C:/Users/ZhangYinhang/ES_F_data.csv")
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values('Date').reset_index(drop=True)
df['log_return'] = np.log(df['Close_ES=F'] / df['Close_ES=F'].shift(1))
df = df.dropna().reset_index(drop=True)

# === 2. Split training and testing sets ===
initial_end = pd.to_datetime("2024-12-23")
rolling_start = pd.to_datetime("2024-12-24")
train_df = df[df['Date'] <= initial_end].copy()
test_df = df[df['Date'] >= rolling_start].copy()

# === 3. Function to identify regimes by mean and std ===
def identify_states_by_mean_std(model):
    mu = model.means_.flatten()
    sigma = np.sqrt(np.array([np.diag(cov)[0] for cov in model.covars_]))
    labels = pd.DataFrame({'state': range(len(mu)), 'mu': mu, 'sigma': sigma})
    labels = labels.sort_values(by='mu')
    labels['regime'] = ['Bear', 'Neutral', 'Bull']
    return labels.set_index('state')['regime'].to_dict()

# === 4. Strategy simulation ===
portfolio = []
portfolio_dates = []
window = 10
test_idx = 0

while test_idx + window <= len(test_df):
    # Fit a new HMM model on the current training set
    model = GaussianHMM(n_components=3, covariance_type="diag", n_iter=100, random_state=9999)
    model.fit(train_df['log_return'].values.reshape(-1, 1))
    state_map = identify_states_by_mean_std(model)

    pred_slice = test_df.iloc[test_idx:test_idx + window].copy()
    pred_returns = pred_slice['log_return'].values.reshape(-1, 1)
    pred_states = model.predict(pred_returns)
    pred_slice['state'] = pred_states
    pred_slice['regime'] = pred_slice['state'].map(state_map)

    # Generate trading signals and compute returns
    for _, row in pred_slice.iterrows():
        if row['regime'] == 'Bull':
            ret = np.log(row['Close_ES=F'] / row['Open_ES=F'])
        elif row['regime'] == 'Bear':
            ret = np.log(row['Open_ES=F'] / row['Close_ES=F'])
        else:
            ret = 0
        portfolio.append(ret)
        portfolio_dates.append(row['Date'])

    # Expand the training set with the latest window
    train_df = pd.concat([train_df, pred_slice], axis=0).reset_index(drop=True)
    test_idx += window

# === 5. Compute overall strategy performance ===
portfolio = np.array(portfolio)
cum_return = np.exp(np.cumsum(portfolio)) - 1
max_drawdown = np.max(np.maximum.accumulate(cum_return) - cum_return)

print(f"\n✅ Cumulative Return: {cum_return[-1]*100:.2f}%")
print(f"📉 Maximum Drawdown: {max_drawdown*100:.2f}%")

# === 6. Compute performance metrics for windows (excluding flat positions) ===
eval_days = [20, 40, 60, 80, 100]
results = []

for d in eval_days:
    if d > len(portfolio):
        break
    r = portfolio[:d]
    curve = np.cumsum(r)
    cumulative = np.exp(np.sum(r)) - 1
    drawdown = np.max(np.maximum.accumulate(curve) - curve)
    
    # Exclude zero-return (flat) days
    r_nonzero = r[np.abs(r) > 1e-6]
    wins = np.sum(r_nonzero > 0)
    losses = np.sum(r_nonzero < 0)
    total_trades = wins + losses
    win_rate = wins / total_trades if total_trades > 0 else np.nan

    results.append({
        'Trading days': d,
        'CumulativeReturn': f"{cumulative*100:.2f}%",
        'MaxDrawdown': f"{drawdown*100:.2f}%",
        'WinRate': f"{win_rate*100:.2f}%" if not np.isnan(win_rate) else "N/A"
    })

result_df = pd.DataFrame(results)
print("\n📊 Strategy Evaluation Table (excluding flat positions):")
print(result_df.to_string(index=False))


Model is not converging.  Current: 2057.7303254178382 is not greater than 2057.8029443891514. Delta is -0.07261897131320438
Model is not converging.  Current: 2091.2875787421085 is not greater than 2091.3254525816333. Delta is -0.03787383952476375
Model is not converging.  Current: 2126.0457397080204 is not greater than 2126.0678043206763. Delta is -0.022064612655867677
Model is not converging.  Current: 2160.0909615639052 is not greater than 2160.1709398257703. Delta is -0.07997826186510792
Model is not converging.  Current: 2179.730765584426 is not greater than 2179.778198108316. Delta is -0.047432523889710865
Model is not converging.  Current: 2216.174319416506 is not greater than 2216.174442027581. Delta is -0.0001226110748575593
Model is not converging.  Current: 2232.717822784037 is not greater than 2232.7266978489038. Delta is -0.008875064866970206
Model is not converging.  Current: 2252.4124284850936 is not greater than 2252.545588831375. Delta is -0.1331603462813291
Model is n


✅ Cumulative Return: 7.20%
📉 Maximum Drawdown: 9.89%

📊 Strategy Evaluation Table (excluding flat positions):
 Trading days CumulativeReturn MaxDrawdown WinRate
           20            3.58%       1.58%  60.00%
           40            2.65%       1.58%  50.00%
           60            1.72%       4.92%  50.00%
           80            3.75%       9.24%  51.52%
          100            7.55%       9.24%  52.78%
